# Model Training

In [1]:
#%pip install pandas
#!pip3 install scikit-learn
#%pip install matplotlib

import sys
import pandas as pd
import sqlite3
import pathlib

sys.path.append("model-training") 
import modelling_functions as mf
import training_config as tc

In [2]:
# Get to the root directory
project_root = pathlib.Path().absolute().parent.parent

# Now construct the relative path to your SQLite database
db_path = project_root / "data" / "footy-tipper-db.sqlite"

# Connect to the SQLite database
con = sqlite3.connect(str(db_path))

# Read the data into a pandas dataframe (assuming the table is named 'my_table')
train_df = pd.read_sql_query("SELECT * FROM train_df", con)

# Don't forget to close the connection
con.close()

In [3]:
mf.training_pipeline(train_df, tc.estimator, tc.outcome_var, tc.predictors)

/opt/homebrew/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


ValueError: Number of classes in y_true not equal to the number of columns in 'y_score'

In [6]:
import numpy as np

print("Unique values in target:", np.unique(train_df[tc.outcome_var]))
print("Target variable shape:", train_df[tc.outcome_var].shape)
train_df[tc.outcome_var]

Unique values in target: [0 1 2]
Target variable shape: (687,)


0      2
1      2
2      1
3      2
4      2
      ..
682    2
683    2
684    1
685    2
686    1
Name: home_team_result, Length: 687, dtype: int64